In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import webbrowser

In [0]:

html = urlopen("http://www.kpfu.ru")
kpfu_main = BeautifulSoup(html, "html.parser")
inst_sites_div = kpfu_main.findAll('div', {'class' : 'institutes'})[10]
inst_site_url_list = [i.a['href'] for i in inst_sites_div.findAll('div')]
inst_list = [[i.a.get_text().strip(),] for i in inst_sites_div.findAll('div')]
for i in range(len(inst_site_url_list)):
    if not ('Институт' in inst_list[i][0] or 'Высшая школа' in inst_list[i][0] or 'факультет' in inst_list[i][0]):
        continue
    inst_list[i].append([])
    cur_inst_site_url = inst_site_url_list[i]
    html_cur_inst_site_url = urlopen(cur_inst_site_url)
    bs_cur_inst_site_url = BeautifulSoup(html_cur_inst_site_url, "html.parser")
    left_menu = bs_cur_inst_site_url.findAll('ul', {'class' : 'menu_list_left'})[0]
    struct_cur_inst_link_element = left_menu.find(lambda tag : tag.name == 'a' and ('Структура' in tag.text or 'Об институте' in tag.text or 'Институт ' in tag.text or 'О нас' in tag.text))
    struct_cur_inst_url = 'not found'
    if struct_cur_inst_link_element != None:
        struct_cur_inst_url = struct_cur_inst_link_element['href']
    print('Parsing ' + struct_cur_inst_url + '   ...')
    if struct_cur_inst_url == 'not found':
        inst_list[i].append([])
        continue
    html_cur_inst_struct = urlopen(struct_cur_inst_url)
    bs_cur_inst_struct = BeautifulSoup(html_cur_inst_struct, "html.parser")
    bs_cur_inst_kaf = bs_cur_inst_struct.findAll(lambda tag : tag.name == 'a' and ('Кафедра' in tag.text or 'Кафедpа' in tag.text) or (tag.name == 'a' and tag.get('style') == 'color: #012a77;font-size: 16px;font-weight: bold;text-decoration:none;' and 'школа' in tag.text))
    if bs_cur_inst_kaf == []:
        try:
            bs_cur_inst_kaf = (BeautifulSoup(urlopen(bs_cur_inst_struct.find('ul', {'class' : 'menu_list_left'}).find(lambda tag: tag.name == 'a' and 'Кафедры' in tag.text)['href']), "html.parser")).findAll(lambda tag : tag.name == 'a' and ('Кафедра' in tag.text))
        except:
            pass
    kaf_list = [[t.get_text().strip(), t.get('href')] for t in bs_cur_inst_kaf]
    for k in kaf_list:
        bs_kaf_page = BeautifulSoup(urlopen(k[1]), "html.parser")
        bs_kaf_sostav_url = bs_kaf_page.find('ul', {'class' : 'menu_list_left'}).find(lambda tag: tag.name == 'a' and ('Коллектив кафедры' in tag.text or 'Сотрудники' in tag.text or 'Состав' in tag.text or 'Список сотрудников' in tag.text))
        if bs_kaf_sostav_url != None:
            kaf_sostav_url = bs_kaf_sostav_url['href']
        else:
            k.append(0)
            continue
        bs_kaf_sostav_iframe = BeautifulSoup(urlopen(kaf_sostav_url), "html.parser").find('iframe')
        if bs_kaf_sostav_iframe == None:
            k.append(0)
            continue
        kaf_sostav_iframe_src = bs_kaf_sostav_iframe['src']
        bs_kaf_workers = BeautifulSoup(urlopen(kaf_sostav_iframe_src), "html.parser").findAll('span', {'class' : 'fio'})
        kaf_workers_num = 0
        if bs_kaf_workers != None:
            kaf_workers_num = len(bs_kaf_workers)
        k.append(kaf_workers_num)
    inst_list[i].append(kaf_list)
print(inst_list)